In [110]:
import pyspark
import pandas as pd

In [111]:
from pymongo import MongoClient, errors
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.classification import LogisticRegression

In [112]:
while True:
    try:
        client = MongoClient("mongodb://bigdata-mongodb-04.virtual.uniandes.edu.co:8087/", retryWrites=False, serverSelectionTimeoutMS=10, connectTimeoutMS=20000)
        client.server_info() # force connection on a request as the
    except errors.ServerSelectionTimeoutError as err:
        print(err)
    finally:
        if client.server_info():            
            break

database = client["Grupo03"]
collection = database["COL_dataset"]
query = {}
query["emocion"] = {

    u"$ne": u""

}
query["$and"] = [

    {

        u"emocion": {

            u"$exists": True

        }

    }

]

cursor = collection.find(query)
data = []

for doc in cursor:
    try:
        data.append([doc['id'], doc['user'].lower(), doc['tweet'].lower(), doc['id_reply_or_quote'],
                     doc['user_replier'].lower(), doc['reply_or_quote'].lower(), doc['emocion'], 
                     doc['tendencia'], doc['coherencia']])
        
    except KeyError as e:        
        data.append([doc['id'], doc['user'].lower(), doc['tweet'].lower(), doc['id_reply_or_quote'],
                     doc['user_replier'].lower(), doc['reply_or_quote'].lower(), doc['emocion'], doc['tendencia'], ''])
        continue

    
client.close()
   
print(len(data))


721


In [113]:


df = pd.DataFrame(data,columns=['id', 'user', 'tweet', 'id_reply_or_quote', 'user_replier',
                                'reply_or_quote', 'emocion', 'tendencia', 'coherencia'])
df.head()
clean_df = df.drop(['id', 'user', 'tweet', 'id_reply_or_quote', 'user_replier',
        'tendencia', 'coherencia'], 1)
clean_df.head()


,reply_or_quote,emocion
0,@claudialopez @infopresidencia @bogota claudia...,3
1,@claudialopez @infopresidencia @bogota dios la...,1
2,@claudialopez @infopresidencia @bogota no clau...,3
3,@claudialopez @infopresidencia @bogota esas ba...,3
4,@claudialopez @infopresidencia @bogota ayuda!\...,2


In [117]:
#clean_df.groupby('emocion')['reply_or_quote'].mean()
#clean_df['count'] = clean_df.groupby(['emocion'])['reply_or_quote'].transform('count')
#df.first()
#df.head(50)


DataError: No numeric types to aggregate